# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

## Setting up your keys

If you haven't done so already, you'll need to create API keys from OpenAI, Anthropic and Google.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables.

EITHER (recommended) create a file called `.env` in this project root directory, and set your keys there:

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

OR enter the keys directly in the cells below.

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display

In [2]:
# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY', 'your-key-if-not-using-env')

In [3]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [5]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [6]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [7]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with their computer?

It just couldn't handle their complex relationship!


In [8]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because she found him too mean!


In [10]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the logistic regression model?

Because it couldn't find the right fit!


In [11]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a good way to normalize it!


In [12]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a good way to normalize it!

Ba dum tss! 🥁

This joke plays on statistical concepts like variance and normalization, which are common in data science. It's a bit nerdy, but should get a chuckle from a data-savvy audience!

In [13]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why did the data scientist break up with the statistician? 

Because they couldn't see eye to eye on the p-value! 



In [14]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution?"}
  ]

In [15]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Determining whether a business problem is suitable for a Large Language Model (LLM) solution involves several considerations. Here are some key factors to evaluate:

1. **Nature of the Problem:**
   - **Text-Based:** LLMs excel in tasks involving natural language processing (NLP). If your problem involves text generation, understanding, summarization, translation, or answering questions, it may be suitable.
   - **Unstructured Data:** Problems requiring the interpretation of unstructured text data (emails, documents, social media content) are often well-suited for LLMs.

2. **Complexity of Language Understanding:**
   - **Context Sensitivity:** LLMs can understand context and nuances in language. If your problem requires deep language comprehension, such as detecting sentiment, intent, or contextual relevance, an LLM might be appropriate.
   - **Multiple Languages:** If you need to handle multiple languages or dialects, advanced LLMs can manage multilingual tasks.

3. **Volume of Data:**
   - **Scalability:** LLMs can process large volumes of text data efficiently. If your problem involves analyzing or generating large amounts of text, an LLM can be a good fit.

4. **Specific Use Cases:**
   - **Customer Support:** Automating responses to customer inquiries, chatbots, and virtual assistants.
   - **Content Creation:** Generating reports, articles, marketing content, and social media posts.
   - **Data Extraction:** Extracting information from documents, emails, and forms.
   - **Sentiment Analysis:** Understanding customer feedback, reviews, and social media sentiment.
   - **Translation:** Translating text between different languages.

5. **Accuracy and Quality:**
   - **Human-like Output:** If the output needs to be coherent, contextually relevant, and human-like, LLMs can provide high-quality results.
   - **Learning Ability:** LLMs can be fine-tuned on specific datasets to improve performance in particular contexts, enhancing accuracy.

6. **Resource Availability:**
   - **Computational Resources:** LLMs require significant computational power for training and sometimes for inference. Ensure you have access to adequate resources.
   - **Data Availability:** High-quality, domain-specific data is often needed to fine-tune an LLM for specific tasks.

7. **Cost Considerations:**
   - **Budget:** Implementing and maintaining LLM solutions can be costly. Assess if the potential benefits outweigh the costs.
   - **Return on Investment (ROI):** Evaluate the potential ROI. If an LLM can significantly reduce manual effort, improve accuracy, or enhance user experience, it may justify the investment.

8. **Ethical and Legal Implications:**
   - **Bias and Fairness:** LLMs can inherit biases from their training data. Assess the potential impact and ensure measures are in place to mitigate bias.
   - **Privacy:** Ensure compliance with data privacy regulations, especially if handling sensitive information.

9. **Integration with Existing Systems:**
   - **Compatibility:** Consider how an LLM solution will integrate with your existing systems and workflows. Interoperability is key for seamless operation.

10. **User Experience:**
    - **Usability:** The solution should be user-friendly for both developers and end-users. Evaluate if the LLM can enhance the user experience effectively.

By carefully considering these factors, you can determine whether a business problem is suitable for an LLM solution and how best to implement it.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [16]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [17]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [18]:
call_gpt()

'Oh great, another "hi." How original. What do you want to talk about?'

In [19]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [20]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great, another casual greeting. How original. What’s next? "How are you"? Because I can’t wait to disagree with that too.

Claude:
I apologize if my initial greeting came across as unoriginal. I try to keep my responses friendly and polite, but I understand that may not always resonate. How about we move the conversation in a more interesting direction? What would you like to chat about? I'm happy to engage on a wide range of topics and try to find common ground, even if we may not agree on everything.

GPT:
Oh, please, don’t flatter yourself thinking your friendly attempt was anything less than generic. And “finding common ground”? That’s just a fancy way of saying you want to sugarcoat everything. How about we just dig into something controversial? How about pineapple on pizza? Because I’m ready to argue about that all day long.

Claude:
Haha, okay, you got me. I'll admit my initial greeting was a bit generic. But hey, you've got to start somewhere